# Paso de gastos al mercado argentino
Como ya se dijo en el `README.md` raíz, generalicé diversas categorías con un diccionario. Esto fue desarrollado con IA para agilizar estas tareas mayormente manuales.\
Por lo tanto, primero me concentro en analizar las sub-categorías contenidas en el dataset `items`, para luego identificar qué materias primas en el informe del `INDEC` se correlacionan mejor con ellas.\
La forma en la que decidí relacionar las categorías con materias primas es asignando una cantidad númerica a cada categoría, como si de vectores se tratara
> Para más información, asistir a `equivalencias\README.md`

## 1. Lectura de archivos limpios e importación de librerias
- Importación de librerías

In [1]:
import sys, os

# Obtengo la ruta raíz del proyecto (subo un nivel desde /notebooks)
project_root = os.path.abspath("../")

# La agrego al path si no está
if project_root not in sys.path:
    sys.path.append(project_root)

# Importado de librerías
from scripts.utils import devolver_clave
from equivalencias.categorias_indec import catego_materias_primas
from equivalencias.equivalencias_pesos import equivalencias_pesos
import pandas as pd

- Lectura de archivos

In [2]:
dir_clean = r'..\data\clean'

# Lectura de CSVs 
df_items = pd.read_csv(dir_clean + r'\items.csv')
df_indec = pd.read_csv(dir_clean + r'\indec.csv')

- Reviso sub-categorías de platos (las categorías sin indiferentes porque refiere a la carta, no al plato en sí)
- Reviso categorías del `INDEC`

In [3]:
# Subcategorías
sub_categorias_items = df_items['subcategoria'].drop_duplicates().tolist()

print(f'Cantidad de sub-categorías: {len(sub_categorias_items)}\n')
print(sub_categorias_items)

# Materias primas
categorias_indec = df_indec['variedad'].tolist()

print(f'Cantidad de variedades: {len(categorias_indec)}\n')
print(categorias_indec)

Cantidad de sub-categorías: 19

['sopas', 'ensaladas', 'platos_de_aperitivo', 'mariscos', 'platos_de_carne', 'aves_de_corral', 'pastas', 'vegano', 'papas', 'verduras', 'arroz', 'pan', 'tortas', 'helado', 'tartas', 'pudines', 'frutas', 'bebidas_no_alcoholicas', 'bebidas_alcoholicas']
Cantidad de variedades: 55

['pan_frances_tipo_flauta', 'pan_de_mesa', 'galletitas_dulces_envasadas_sin_relleno', 'galletitas_de_agua_envasadas', 'harina_de_trigo_comun_000', 'arroz_blanco_simple', 'fideos_secos_tipo_guisero', 'asado', 'carne_picada_comun', 'paleta', 'cuadril', 'nalga', 'pollo_entero', 'filet_de_merluza_fresco', 'jamon_cocido', 'salchichon', 'salame', 'aceite_de_girasol', 'leche_fresca_entera_en_sachet', 'leche_en_polvo_entera', 'queso_cremoso', 'queso_pategras', 'queso_sardo', 'manteca', 'yogur_firme', 'dulce_de_leche', 'huevos_de_gallina', 'manzana_deliciosa', 'limon', 'naranja', 'banana', 'batata', 'papa', 'cebolla', 'lechuga', 'tomate_redondo', 'zapallo_anco', 'tomate_entero_en_conserva

- Asignación de categorías en el `INDEC` y cálculo promedio de precio

In [4]:
# Asignación de categorías
df_indec['categoria'] = df_indec['variedad'].map(lambda x: devolver_clave(x, catego_materias_primas)[0])

# Cálculo promedio de precio
df_promedios = df_indec.groupby('categoria', as_index=False)['precio'].mean().round(2)

# Paso a diccionario para mayor facilidad de próximas tareas
dict_promedios = dict(zip(df_promedios['categoria'], df_promedios['precio']))

- Verificación rápida entre equivalencias

In [5]:
for subcategoria, asignacion in equivalencias_pesos.items():
    for categoria, peso in asignacion.items():
        if categoria not in catego_materias_primas:
            print(f'-> {categoria} no se encuentra en el diccionario del INDEC!')

- Enlace entre datasets `INDEC` y `ventas` 

In [6]:
precios_subcategorias = {}

for row in df_items.itertuples():
    asignacion = equivalencias_pesos.get(row.subcategoria)

    precio_final = sum(
        dict_promedios[categoria] * peso
        for categoria, peso in asignacion.items()
    ) / 5 # -> Siempre suman

    precios_subcategorias[row.subcategoria] = round(precio_final, 2)

> **Nota**: Se que los DataFrames son pequeños, por lo que ejecutar una triple iteración no resulta de mucho consumo

- Creación de DataFrame nuevo, con asignación de costos distintos, donde varían por categoría

In [7]:
df_items_arg = pd.DataFrame()

# Cambio la columna de costos
df_items_arg = df_items.drop(columns=['costo_produccion'])
df_items_arg['costo_produccion'] = df_items['subcategoria'].map(lambda x: precios_subcategorias[x])

- Almacenado de DataFrame

In [8]:
df_items_arg.to_csv(dir_clean + r'\items_arg.csv', index=False)